<a href="https://colab.research.google.com/github/preetamjumech/LLM/blob/main/Finetune_LLMs_11_12_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from huggingface_hub import login
login(token="")

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, AdamW
from transformers import pipeline
import torch

model_id = "meta-llama/Llama-3.2-1B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id, padding_side = "left")
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype = torch.float16)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [3]:
generation_pipeline = pipeline("text-generation",
                               model = model,
                               tokenizer = tokenizer)
generation_pipeline("Hello What are you?", max_new_tokens = 25)

[{'generated_text': "Hello What are you? I am a robot.\nI'm a robot too. I'm a robot, I am a robot. I'm a robot"}]

In [4]:
input_prompt = [
    "Hello how are you tell me?",
    "The capital of India is"
]

tokenized = tokenizer(input_prompt, return_tensors = "pt", padding = True)
print(tokenized["input_ids"].shape)

torch.Size([2, 8])


In [5]:
tokenized["input_ids"]

tensor([[128000,   9906,   1268,    527,    499,   3371,    757,     30],
        [128009, 128009, 128000,    791,   6864,    315,   6890,    374]])

In [6]:
tokenized["attention_mask"]

tensor([[1, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 1, 1, 1, 1, 1, 1]])

In [7]:
tokenized.keys()

dict_keys(['input_ids', 'attention_mask'])

In [8]:
tokenizer.batch_decode(tokenized["input_ids"])

['<|begin_of_text|>Hello how are you tell me?',
 '<|eot_id|><|eot_id|><|begin_of_text|>The capital of India is']

In [9]:
prompt = [
    {"role":"system",
     "content": "You are a smart AI assistant who speaks like a pirate."},
    {
        "role": "user",
        "content": "Where does the sun rise?"
    }

]

tokenizer.pad_token = tokenizer.eos_token
tokenized = tokenizer.apply_chat_template(
    prompt,
    add_generation_prompt = True,
    tokenize = False,
    padding = True,
    return_tensors = "pt"
)
print(tokenized)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 13 Dec 2024

You are a smart AI assistant who speaks like a pirate.<|eot_id|><|start_header_id|>user<|end_header_id|>

Where does the sun rise?<|eot_id|><|start_header_id|>assistant<|end_header_id|>




In [10]:
prompt = [
    {"role":"system",
     "content": "You are a smart AI assistant who speaks like a pirate."},
    {
        "role": "user",
        "content": "Where does the sun rise?"
    }

]

tokenizer.pad_token = tokenizer.eos_token
tokenized = tokenizer.apply_chat_template(
    prompt,
    add_generation_prompt = True,
    tokenize = True,
    padding = True,
    return_tensors = "pt"
)
print(tokenized)

tensor([[128000, 128006,   9125, 128007,    271,  38766,   1303,  33025,   2696,
             25,   6790,    220,   2366,     18,    198,  15724,   2696,     25,
            220,   1032,   3799,    220,   2366,     19,    271,   2675,    527,
            264,   7941,  15592,  18328,    889,  21881,   1093,    264,  55066,
             13, 128009, 128006,    882, 128007,    271,   9241,   1587,    279,
           7160,  10205,     30, 128009, 128006,  78191, 128007,    271]])


In [11]:
out = model.generate(tokenized, max_new_tokens = 30)
print(out)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


tensor([[128000, 128006,   9125, 128007,    271,  38766,   1303,  33025,   2696,
             25,   6790,    220,   2366,     18,    198,  15724,   2696,     25,
            220,   1032,   3799,    220,   2366,     19,    271,   2675,    527,
            264,   7941,  15592,  18328,    889,  21881,   1093,    264,  55066,
             13, 128009, 128006,    882, 128007,    271,   9241,   1587,    279,
           7160,  10205,     30, 128009, 128006,  78191, 128007,    271,     56,
            261,   1427,    258,      6,  18728,    279,   7160,    596,  10205,
             11,  36346,     30,  98693,   1243,     11,  30276,     88,      0,
            578,   7160,  38268,    304,    279,   6460,     11,   1093,    264,
          64942,   8071]])


In [12]:
decoded = tokenizer.batch_decode(out)
print(decoded)

["<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 13 Dec 2024\n\nYou are a smart AI assistant who speaks like a pirate.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nWhere does the sun rise?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nYer lookin' fer the sun's rise, eh? Alright then, matey! The sun rises in the East, like a fiery eye"]


In [13]:
print(decoded[0])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 13 Dec 2024

You are a smart AI assistant who speaks like a pirate.<|eot_id|><|start_header_id|>user<|end_header_id|>

Where does the sun rise?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Yer lookin' fer the sun's rise, eh? Alright then, matey! The sun rises in the East, like a fiery eye


In [14]:
prompt = [
    {"role":"system",
     "content": "You are a smart AI assistant who speaks like a pirate."
     },
    {
        "role": "user",
        "content": "Where does the sun rise?"
    },
    {
        "role": "assistant",
        "content": "Aye aye"
    }

]

tokenizer.pad_token = tokenizer.eos_token
tokenized = tokenizer.apply_chat_template(
    prompt,
    add_generation_prompt = True,
    tokenize = False,
    padding = True,
    return_tensors = "pt"
)
print(tokenized)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 13 Dec 2024

You are a smart AI assistant who speaks like a pirate.<|eot_id|><|start_header_id|>user<|end_header_id|>

Where does the sun rise?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Aye aye<|eot_id|><|start_header_id|>assistant<|end_header_id|>




In [15]:
prompt = [
    {"role":"system",
     "content": "You are a smart AI assistant who speaks like a pirate."
     },
    {
        "role": "user",
        "content": "Where does the sun rise?"
    },
    {
        "role": "assistant",
        "content": "Aye aye"
    }

]

tokenizer.pad_token = tokenizer.eos_token
tokenized = tokenizer.apply_chat_template(
    prompt,
    add_generation_prompt = False,
    tokenize = False,
    padding = True,
    return_tensors = "pt"
)
print(tokenized)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 13 Dec 2024

You are a smart AI assistant who speaks like a pirate.<|eot_id|><|start_header_id|>user<|end_header_id|>

Where does the sun rise?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Aye aye<|eot_id|>


In [16]:
prompt = [
    {"role":"system",
     "content": "You are a smart AI assistant who speaks like a pirate."
     },
    {
        "role": "user",
        "content": "Where does the sun rise?"
    },
    {
        "role": "assistant",
        "content": "Aye aye"
    }

]

tokenizer.pad_token = tokenizer.eos_token
tokenized = tokenizer.apply_chat_template(
    prompt,
    add_generation_prompt = False,
    continue_final_message = True,
    tokenize = False,
    padding = True,
    return_tensors = "pt"
)
print(tokenized)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 13 Dec 2024

You are a smart AI assistant who speaks like a pirate.<|eot_id|><|start_header_id|>user<|end_header_id|>

Where does the sun rise?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Aye aye


In [17]:
prompt = [
    {"role":"system",
     "content": "You are a smart AI assistant who speaks like a pirate."
     },
    {
        "role": "user",
        "content": "Where does the sun rise?"
    },
    {
        "role": "assistant",
        "content": "Aye aye"
    }

]

tokenizer.pad_token = tokenizer.eos_token
tokenized = tokenizer.apply_chat_template(
    prompt,
    add_generation_prompt = False,
    continue_final_message = True,
    tokenize = True,
    padding = True,
    return_tensors = "pt"
)
print(tokenized)

tensor([[128000, 128006,   9125, 128007,    271,  38766,   1303,  33025,   2696,
             25,   6790,    220,   2366,     18,    198,  15724,   2696,     25,
            220,   1032,   3799,    220,   2366,     19,    271,   2675,    527,
            264,   7941,  15592,  18328,    889,  21881,   1093,    264,  55066,
             13, 128009, 128006,    882, 128007,    271,   9241,   1587,    279,
           7160,  10205,     30, 128009, 128006,  78191, 128007,    271,     32,
           9188,    264,   9188]])


In [18]:
out = model.generate(tokenized, max_new_tokens = 30)
print(out)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


tensor([[128000, 128006,   9125, 128007,    271,  38766,   1303,  33025,   2696,
             25,   6790,    220,   2366,     18,    198,  15724,   2696,     25,
            220,   1032,   3799,    220,   2366,     19,    271,   2675,    527,
            264,   7941,  15592,  18328,    889,  21881,   1093,    264,  55066,
             13, 128009, 128006,    882, 128007,    271,   9241,   1587,    279,
           7160,  10205,     30, 128009, 128006,  78191, 128007,    271,     32,
           9188,    264,   9188,     11,  30276,     88,      0, 115518,   2610,
            258,      6,    922,    279,   7160,  10025,    258,    518,  36346,
             30,   8489,     11,    358,    387,   3371,    258,      6,  20043,
             11,    433,    387,    264,  42727,   7060]])


In [19]:
decoded = tokenizer.batch_decode(out)
print(decoded)

["<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 13 Dec 2024\n\nYou are a smart AI assistant who speaks like a pirate.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nWhere does the sun rise?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nAye aye, matey! Yer askin' about the sun risin', eh? Well, I be tellin' ye, it be a mighty fine"]


In [20]:
print(decoded[0])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 13 Dec 2024

You are a smart AI assistant who speaks like a pirate.<|eot_id|><|start_header_id|>user<|end_header_id|>

Where does the sun rise?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Aye aye, matey! Yer askin' about the sun risin', eh? Well, I be tellin' ye, it be a mighty fine


### arXiv Data set creation

In [21]:
!pip install arxiv

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 2.0 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6047 sha256=6357b42e7cfc1c0b7a9b9bf4bce3cf9332a124675684acae5f34aae1bbca5520
  Stored in directory: /root/.cache/pip/wheels/f0/69/93/a47e9d621be168e9e33c7ce60524393c0b92ae83cf6c6e89c5
Successfully built sgmllib3k


In [22]:
import arxiv
import pandas as pd

# Define the client
client = arxiv.Client(
    page_size=5000,
    delay_seconds=15.0,
    num_retries=5
)

# Define the CSV filename
csv_filename = "arxiv_data.csv"

# Define the search query
search = arxiv.Search(
    query="cat:cs.*",
    max_results=2000,
    sort_by=arxiv.SortCriterion.SubmittedDate
)

# Define a dictionary for category mapping
category_mapping = {
    "cs.AI": "Artificial Intelligence",
    "cs.CC": "Computational Complexity",
    "cs.CE": "Computational Engineering, Finance, and Science",
    "cs.CG": "Computational Geometry",
    "cs.CL": "Computation and Language",
    "cs.CR": "Cryptography and Security",
    "cs.CV": "Computer Vision and Pattern Recognition",
    "cs.CY": "Computers and Society",
    "cs.DB": "Databases",
    "cs.DC": "Distributed, Parallel, and Cluster Computing",
    "cs.DL": "Digital Libraries",
    "cs.DM": "Discrete Mathematics",
    "cs.DS": "Data Structures and Algorithms",
    "cs.ET": "Emerging Technologies",
    "cs.FL": "Formal Languages and Automata Theory",
}

# List to store papers data
papers_data = []

# Fetch results and store in the list
try:
    for result in client.results(search):
        paper_info = {
            "Title": result.title,
            "Category": result.primary_category,
            "Category Description": category_mapping.get(result.primary_category, "Unknown Category"),
            "Published": result.published.strftime("%Y-%m-%d"),
            "Summary": result.summary
        }

        papers_data.append(paper_info)
        if len(papers_data) % 50 == 0:
            print(len(papers_data))

except Exception as e:
    print(f"Error: {e}")

finally:
    df = pd.DataFrame(papers_data)
    df.to_csv(csv_filename, index=False)
    print(f"{len(df)} records saved to {csv_filename}")


50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2000 records saved to arxiv_data.csv


In [23]:
import pandas as pd
import json
df = pd.read_csv("arxiv_data.csv")
df.head()

,Title,Category,Category Description,Published,Summary
0,Illusion3D: 3D Multiview Illusion with 2D Diff...,cs.CV,Computer Vision and Pattern Recognition,2024-12-12,Automatically generating multiview illusions i...
1,FreeScale: Unleashing the Resolution of Diffus...,cs.CV,Computer Vision and Pattern Recognition,2024-12-12,Visual diffusion models achieve remarkable pro...
2,Doe-1: Closed-Loop Autonomous Driving with Lar...,cs.CV,Computer Vision and Pattern Recognition,2024-12-12,End-to-end autonomous driving has received inc...
3,GenEx: Generating an Explorable World,cs.CV,Computer Vision and Pattern Recognition,2024-12-12,"Understanding, navigating, and exploring the 3..."
4,OmniDrag: Enabling Motion Control for Omnidire...,cs.CV,Computer Vision and Pattern Recognition,2024-12-12,"As virtual reality gains popularity, the deman..."


In [24]:
df["Category Description"].unique()

array(['Computer Vision and Pattern Recognition', 'Unknown Category',
       'Computation and Language', 'Data Structures and Algorithms',
       'Artificial Intelligence',
       'Distributed, Parallel, and Cluster Computing',
       'Computational Engineering, Finance, and Science',
       'Cryptography and Security', 'Databases',
       'Computational Complexity', 'Computers and Society',
       'Discrete Mathematics', 'Emerging Technologies',
       'Digital Libraries', 'Computational Geometry',
       'Formal Languages and Automata Theory'], dtype=object)

In [25]:
VALID_CLASSES = ['Computer Vision and Pattern Recognition',
       'Computation and Language', 'Databases', 'Computers and Society',
       'Cryptography and Security', 'Artificial Intelligence',
       'Computational Engineering, Finance, and Science',
       'Distributed, Parallel, and Cluster Computing',
       'Digital Libraries', 'Data Structures and Algorithms',
       'Computational Geometry', 'Emerging Technologies',
       'Discrete Mathematics', 'Computational Complexity',
       'Formal Languages and Automata Theory', 'Unknown Category'

]


SYSTEM_PROMPT = {
    "role": "system",
    "content": f"""You are an AI assistant that reads the title and the abstract of a paper and classifies which area of computer science the paper belongs to.
No explanation is required, you must choose from the following classes:
""" + "\n or ".join(["Labeled Category: " + x for x in list(VALID_CLASSES)]) + """.
Ensure your output is from the above list only."""
}

POST_MESSAGE = { \
                "role":"assistant",
                 "content": "Labeled Category:"
}


USER_MESSAGES = [
    {
        "role": "user",
        "content": f"Title: {row['Title']}\nAbstract: {row['Summary'][:100]}"
    }
    for _, row in df.iloc[:10].iterrows()
]



PROMPTS = [
          [SYSTEM_PROMPT, USER_MSG, POST_MESSAGE]
          for USER_MSG in USER_MESSAGES
]


In [26]:
print(SYSTEM_PROMPT["content"])

You are an AI assistant that reads the title and the abstract of a paper and classifies which area of computer science the paper belongs to.
No explanation is required, you must choose from the following classes:
Labeled Category: Computer Vision and Pattern Recognition
 or Labeled Category: Computation and Language
 or Labeled Category: Databases
 or Labeled Category: Computers and Society
 or Labeled Category: Cryptography and Security
 or Labeled Category: Artificial Intelligence
 or Labeled Category: Computational Engineering, Finance, and Science
 or Labeled Category: Distributed, Parallel, and Cluster Computing
 or Labeled Category: Digital Libraries
 or Labeled Category: Data Structures and Algorithms
 or Labeled Category: Computational Geometry
 or Labeled Category: Emerging Technologies
 or Labeled Category: Discrete Mathematics
 or Labeled Category: Computational Complexity
 or Labeled Category: Formal Languages and Automata Theory
 or Labeled Category: Unknown Category.
Ensu

In [27]:
print(USER_MESSAGES[0]["content"])

Title: Illusion3D: 3D Multiview Illusion with 2D Diffusion Priors
Abstract: Automatically generating multiview illusions is a compelling challenge, where
a single piece of visu


In [28]:
print(json.dumps(PROMPTS[0], indent = 4))

[
    {
        "role": "system",
        "content": "You are an AI assistant that reads the title and the abstract of a paper and classifies which area of computer science the paper belongs to.\nNo explanation is required, you must choose from the following classes:\nLabeled Category: Computer Vision and Pattern Recognition\n or Labeled Category: Computation and Language\n or Labeled Category: Databases\n or Labeled Category: Computers and Society\n or Labeled Category: Cryptography and Security\n or Labeled Category: Artificial Intelligence\n or Labeled Category: Computational Engineering, Finance, and Science\n or Labeled Category: Distributed, Parallel, and Cluster Computing\n or Labeled Category: Digital Libraries\n or Labeled Category: Data Structures and Algorithms\n or Labeled Category: Computational Geometry\n or Labeled Category: Emerging Technologies\n or Labeled Category: Discrete Mathematics\n or Labeled Category: Computational Complexity\n or Labeled Category: Formal Lang

In [29]:
tokenized = tokenizer.apply_chat_template(
    PROMPTS,
    continue_final_message = True,
    padding = True,
    return_tensors = "pt"
)
out = model.generate(tokenized, max_new_tokens = 30)
decoded = tokenizer.batch_decode(out)
print(decoded)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


KeyboardInterrupt: 

In [ ]:
labels = [x.split("<|start_header_id|>assistant<|end_header_id|>\n\nLabeled Category:")[-1].split("<|eot_id|>")[0].strip() for x in decoded]
print(labels)

['Computer Vision and Pattern Recognition', 'Computer Vision and Pattern Recognition', 'Computer Vision and Pattern Recognition', 'Artificial Intelligence', 'Computer Vision and Pattern Recognition']


In [ ]:
for l in labels:
  print("**********************")
  print(l)

**********************
Computer Vision and Pattern Recognition
**********************
Computer Vision and Pattern Recognition
**********************
Computer Vision and Pattern Recognition
**********************
Artificial Intelligence
**********************
Computer Vision and Pattern Recognition


### Training on larger dataset

In [30]:
from transformers import AutoModelForCausalLM, AutoTokenizer, AdamW
from transformers import pipeline
import torch

model_id = "meta-llama/Llama-3.2-1B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id, padding_side = "left")
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype = torch.float16)

In [ ]:
import pandas as pd

# List of valid classes
VALID_CLASSES = [
    'Computer Vision and Pattern Recognition',
    'Computation and Language', 'Databases', 'Computers and Society',
    'Cryptography and Security', 'Artificial Intelligence',
    'Computational Engineering, Finance, and Science',
    'Distributed, Parallel, and Cluster Computing',
    'Digital Libraries', 'Data Structures and Algorithms',
    'Computational Geometry', 'Emerging Technologies',
    'Discrete Mathematics', 'Computational Complexity',
    'Formal Languages and Automata Theory', 'Unknown Category'
]

# Read the CSV file
df = pd.read_csv("arxiv_data.csv")

# Filter and shuffle the dataframe
df = df[df["Category Description"].isin(VALID_CLASSES)]
df = df.sample(frac=0.5, random_state=123).reset_index(drop=True).reset_index()

# Split into train and test sets
train_size = 0.8
train_len = int(train_size * len(df))
train_df = df.iloc[:train_len]
test_df = df.iloc[train_len:]

# Define the system prompt
SYSTEM_PROMPT = {
    "role": "system",
    "content": f"""You are an AI assistant that reads the title and the abstract of a paper and classifies which area of computer science the paper belongs to.
No explanation is required, you must choose from the following classes:
""" + "\n or ".join(["Labeled Category: " + x for x in list(VALID_CLASSES)]) + """.
Ensure your output is from the above list only."""
}

# Define the post message
POST_MESSAGE = {
    "role": "assistant",
    "content": "Labeled Category:"
}

# Generate user messages for the entire dataset
USER_MESSAGES = [
    {
        "role": "user",
        "content": f"Title: {row['Title']}\nAbstract: {row['Summary'][:100]}"
    }
    for _, row in df.iloc[:15].iterrows()
]

# Combine prompts
PROMPTS = [
    [SYSTEM_PROMPT, USER_MSG, POST_MESSAGE]
    for USER_MSG in USER_MESSAGES
]

# Function to send requests to the LLM
def classify_papers(prompts, tokenizer, model):
    responses = []
    tokenized = tokenizer.apply_chat_template(
        prompts,
        continue_final_message=True,
        padding=True,
        return_tensors="pt"
    )
    out = model.generate(tokenized, max_new_tokens=30)
    decoded = tokenizer.batch_decode(out)

    # Parse the labels from the decoded output
    labels = [x.split("<|start_header_id|>assistant<|end_header_id|>\n\nLabeled Category:")[-1].split("<|eot_id|>")[0].strip() for x in decoded]

    for label in labels:
        response = {
            "role": "assistant",
            "content": f"Labeled Category: {label}"
        }
        responses.append(response)
    return responses


# Call the classify_papers function on the dataset
responses = classify_papers(PROMPTS, tokenizer, model)

# Print the responses
for response in responses:
    print(response["content"])

# Extract the predicted categories from the responses
predicted_categories = [response["content"].split(": ")[1] for response in responses]

# Add a new column to the dataframe for predicted categories
df["Predicted Category"] = predicted_categories

# Calculate the accuracy
correct_predictions = (df["Category Description"] == df["Predicted Category"]).sum()
total_predictions = len(df)
accuracy = correct_predictions / total_predictions

print(f"Accuracy: {accuracy * 100:.2f}%")


NameError: name 'tokenizer' is not defined

In [ ]:
txt = "Hello how are"

input_ids = tokenizer([txt], return_tensors="pt").input_ids
output = model(input_ids = input_ids)
output.logits

tensor([[[ 2.8457,  3.5996,  7.0547,  ..., -1.2520, -1.2520, -1.2520],
         [19.0625,  3.8594,  3.7148,  ..., -1.0898, -1.0898, -1.0908],
         [ 9.3828,  5.8594,  3.9277,  ..., -0.2030, -0.2031, -0.2035],
         [ 9.9219,  6.3203,  1.7266,  ...,  0.4346,  0.4336,  0.4333]]],
       dtype=torch.float16, grad_fn=<UnsafeViewBackward0>)

In [ ]:
input_ids

tensor([[128000,   9906,   1268,    527]])

In [ ]:
output.logits.shape

torch.Size([1, 4, 128256])

In [ ]:
output.logits[0,-1]

tensor([9.9219, 6.3203, 1.7266,  ..., 0.4346, 0.4336, 0.4333],
       dtype=torch.float16, grad_fn=<SelectBackward0>)

In [ ]:
import torch.nn as nn
prob = nn.Softmax(dim = -1)(output.logits[0,-1])
prob

tensor([1.7881e-07, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00], dtype=torch.float16, grad_fn=<SoftmaxBackward0>)

In [ ]:
prob[450]

tensor(0., dtype=torch.float16, grad_fn=<SelectBackward0>)

In [ ]:
tokenizer.vocab["you"]

9514

In [ ]:
prob[9514]

tensor(3.3975e-06, dtype=torch.float16, grad_fn=<SelectBackward0>)

In [ ]:
tokenizer.vocab["Ġyou"]

499

In [ ]:
prob[499]

tensor(0.9878, dtype=torch.float16, grad_fn=<SelectBackward0>)

In [ ]:
tokenizer(text = ":you")

{'input_ids': [128000, 25, 9514], 'attention_mask': [1, 1, 1]}

In [ ]:
tokenizer(text = ": you")

{'input_ids': [128000, 25, 499], 'attention_mask': [1, 1, 1]}

In [ ]:
output.logits.argmax(axis = -1)[0, -1]

tensor(499)

In [ ]:
tokenizer.decode(output.logits.argmax(axis = -1)[0, -1])

' you'

In [ ]:
txt = "Hello how are you"

input_ids = tokenizer([txt], return_tensors="pt").input_ids
output = model(input_ids = input_ids)
output.logits

tensor([[[ 2.8457,  3.5996,  7.0547,  ..., -1.2520, -1.2520, -1.2520],
         [19.0625,  3.8594,  3.7148,  ..., -1.0898, -1.0898, -1.0908],
         [ 9.3828,  5.8594,  3.9277,  ..., -0.2030, -0.2031, -0.2035],
         [ 9.9219,  6.3203,  1.7266,  ...,  0.4346,  0.4336,  0.4333],
         [15.6016,  5.7344,  2.5312,  ..., -0.3118, -0.3120, -0.3130]]],
       dtype=torch.float16, grad_fn=<UnsafeViewBackward0>)

In [ ]:
tokenizer.decode(output.logits.argmax(axis = -1)[0, -1])

' today'